# Task Sequencer

#### <i>"Sometimes when you innovate, you make mistakes. It is best to admit them quickly, and get on with improving your other innovations."<i> -Steve Jobs

This is a tool which will calculate the optimal order to approach a project where there are several different tasks that need to be done that have some probability of failing.

When we are uncertain about the feasibility of a project, we should strive to get as much information about the difficulty as quickly as possible. This suggests that we should start with tasks with a high failure rate.

The idea behind this approach is and some other useful information is discussed here. https://cs.stanford.edu/~jsteinhardt/ResearchasaStochasticDecisionProcess.html

In [237]:
import ipywidgets
import ipysheet
import pandas as pd
from itertools import permutations
import numpy as np
import warnings
import matplotlib.pyplot as plt
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['lines.linewidth'] = 2.0
from IPython.display import Markdown, display, clear_output, FileLink
from symbulate import RV, Exponential


def printmd(string):
    display(Markdown(string))

def df_from_sheet(sheet):
    '''Converts the sheet to a compact dataframe'''
    df = ipysheet.to_dataframe(sheet)
    df = fill_blanks(df)
    df = trim_df(df)
    df = switch_from_names(df)
    df = nums_to_float(df)
    df = switch_to_names(df)
    return df

def switch_to_names(df):
    '''Converts indicies in the dependency cols
    to their corresponding task names'''
    df.iloc[:,3:] = df.iloc[:,3:].replace(
        dict(zip(df.index + 1, df['Task'])))
    return df

def switch_from_names(df):
    '''Converts the task names  in the dependency 
    cols to their coresponding indicies'''
    df.iloc[:,3:] = df.iloc[:,3:].replace(
        names_to_index_dict(df))
    return df

def names_to_index_dict(df):
    return dict(zip(df['Task'], df.index + 1))

def fill_blanks(df):
    '''Replaces NaN and empty values with 0s'''
    df.fillna(0, inplace=True)
    df.replace(["", "0"], 0, inplace=True)
    return df

def trim_df(df):
    '''Removes rows and columns that are blank (
    or contain zeros)'''
    zero_row_bool = (df.values==0).all(axis=1)
    zero_row_loc = np.where(zero_row_bool)[0]
    if zero_row_loc.size == 0:
        first_zero_row=df.shape[0]
    else:
        first_zero_row = zero_row_loc[0]
    zero_col_bool = (df.values==0).all(axis=0)
    zero_col_loc = np.where(zero_col_bool)[0]
    if zero_col_loc.size == 0:
        first_zero_col = df.shape[1]
    else:
        first_zero_col = zero_col_loc[0]
    return df.iloc[:first_zero_row,:first_zero_col]
    
def nums_to_float(df):
    '''Converts all numerical entries
    to floats'''
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    return df

def build_sheet(task_report="task_sheet.csv",
               rows=10, cols=10):
    '''Creates the task dependency sheet'''
    labels = (['Task', 'Time', 'Probability'] 
              + ["D" + str(i + 1) 
                 for i in range(cols - 3)])
    sheet = sheet_from_df(task_report, rows, 
                          cols, labels)
    if sheet:
        return sheet
    sheet = sheet_from_scratch(rows, cols, labels)
    return sheet

def sheet_from_df(task_report, rows, 
                  cols, labels):
    '''Returns a dataframe from task_report if
    task_report is in the working directory. Otherwise
    returns None.'''
    try:
        sheet = ipysheet.sheet(
            rows=rows,
            columns=cols,
            column_headers=labels)
        df = pd.read_csv(task_report)
        df = rid_zeros(df)
        data = np.empty([rows, cols], dtype=object)
        data[:df.shape[0], :df.shape[1]] = df.values
        ipysheet.cell_range(data)
        return sheet
    except:
        return None

def rid_zeros(df):
    '''Converts the task names  in the dependency 
    cols to their coresponding indicies'''
    df.iloc[:,3:] = df.iloc[:,3:].replace({'0.0' : None})
    return df
    
    
def sheet_from_scratch(rows, cols, labels):
    '''Generates a sheet with all empty values'''
    sheet = ipysheet.sheet(
        rows=rows,
        columns=cols,
        column_headers=labels)
    data = [['' for i in range(cols)] 
            for i in range(rows)]
    ipysheet.cell_range(data)
    return sheet

def gen_perms(df):
    '''Creates a list of all permutations of the
    rows of the dataframe (indexing starting at 1)'''
    rows=df.shape[0]
    return permutations(range(1,rows+1))

def validate_perms(perms, deps):
    '''Returns the rows in perms which dont
    violate a dependency'''
    valid_perms = []
    for perm in perms:
        if check_perm(perm, deps) == True:
            valid_perms.append(perm)
    return valid_perms

def check_perm(perm, deps):
    '''Returns True or False according to whether
    perm violates a dependency'''
    prev_ind = [0]
    for index in perm:
        prev_ind.append(index)
        dep = deps[index-1, :]
        sub_true = all(elem in prev_ind for elem in dep)
        if sub_true is False:
            return False
        elif index == perm[-1]:
            return True

def sort_rates(times, probs):
    '''Returns an order list of the
    failure rates (called lambda in the
    appendix)'''
    return (np.flip(np.argsort(
        1/times*np.log(1/(1-probs)))) + 1)
        
def get_expected_times(perms, df):
    '''Calculates the expected time for each
    order of the tasks according to the
    equation in appendix A'''
    sums = []
    for perm in perms:
        prob_col = df['Probability'][np.array(perm)-1].values.astype(float)
        prob_comp = 1 - prob_col
        prob_mat = expand_to_lower_tri_mat(prob_comp)
        prod_arr = prob_prod_arr(prob_mat, prob_col)
        time_col = df['Time'][np.array(perm) - 1].values.astype(float)
        time_mat = expand_to_lower_tri_mat(time_col)
        sum_arr = sum_times(time_mat)
        sums.append(expected_time_eq(prod_arr,
                                     sum_arr, time_col,
                                     prob_col, prob_comp))
    return sums


def expected_time_eq(prod_arr, sum_arr, time_col, prob_col,
                     prob_comp):
    prob_succ = np.prod(prob_comp)
    total_time = np.sum(time_col)
    exp_time_given_fail_on_task = (sum_arr
                                   - time_col/prob_col
                                   + time_col/np.log(1/(1 - prob_col))
                                   + time_col)
    prob_fail_on_task = prod_arr
    return (prob_succ*total_time
           + np.sum(prob_fail_on_task*exp_time_given_fail_on_task))



def expand_to_lower_tri_mat(arr):
    '''Converts arr to a lower triangular matrix.
    e.g. [1,2,3] => [[1,0,0]
                     [1,2,0]
                     [1,2,3]]'''
    two_d_arr = np.expand_dims((arr), 0)
    square_mat = np.repeat(two_d_arr,
                             repeats=arr.shape[0],
                             axis=0)
    return np.tril(square_mat)
    
def prob_prod_arr(prob_mat, prob_col):
    '''Returns the product array that will satisfy the
    equation in the appendix e.g. 
    prob_mat = [[0.8, 0.0, 0.0],    prob_col = [0.2, 0.3, 0.4]
                [0.8, 0.7, 0.0],
                [0.8, 0.7, 0.6]]
    => prod_arr = column product of( [[0.8, 0.0, 0.0], 
                                     [0.8, 0.7, 0.0],
                                     [0.8, 0.7, 0.6]]                    
                                    +[[0.2, 0.0, 0.0],
                                      [0.0, 0.3, 0.0],
                                      [0.0, 0.0, 0.4]] 
                                    +[[0.0, 1.0, 1,0],
                                      [0.0, 0.0, 1.0],
                                      [0.0, 0.0, 0.0]] )
      which equals [0.2  , 0.24 , 0.224]'''                              
    np.fill_diagonal(prob_mat, 0)
    ones = np.triu(np.ones(np.shape(prob_mat)))
    np.fill_diagonal(ones, 0)
    return np.prod(ones + prob_mat + np.diag(prob_col), 1)

def sum_times(time_mat):
    '''Returns an array of sums from
    a lower triangular matrix after zeroing
    diagonal'''
    np.fill_diagonal(time_mat, 0)
    return np.sum(time_mat, 1)

def adj_transposes(arr):
    """returns a matrix where each row
    is one of the n-1 adjacent transposes"""
    reps_mat = repeat_array_as_matrix(arr, len(arr)-1)
    for i in range(len(arr)-1):
        ith_row = reps_mat[i,:]
        ith_row[i], ith_row[i+1] = ith_row[i+1], ith_row[i]
    return reps_mat

def find_kids(row, arr, deps):
    '''Returns the inidicies of srtd_rates where
    the dependents of task in "row" are located'''
    child_rows = np.argwhere(deps==row)[:,0] + 1
    child_loc = np.where(np.in1d(arr, child_rows))[0]
    return child_loc

def repeat_array_as_matrix(arr, reps):
    '''e.g. array([1,2]) to array([[1,2],
                                    [1,2]])'''
    matrix_head = np.expand_dims(arr, axis=0)
    return np.repeat(matrix_head,
                      repeats=reps,
                        axis=0)

def find_parents(row, arr, deps):
    '''Returns the inidicies of arr where
    the tasks that task in row is dependent on
    are located'''
    dep = deps[row-1] #minus 1 cause index
    parent_rows = dep[np.nonzero(dep)] #rid zeros
    parent_loc = np.where(np.in1d(arr, parent_rows))[0]
    return parent_loc

def calc_disarray(arr, deps):
    '''returns a number according to how disarrayed
    arr is'''
    total_disarray = 0
    for task_loc, task in enumerate(arr):
        task_kids_loc = find_kids(task, arr, deps)
        task_pars_loc = find_parents(task, arr, deps)
        total_disarray += element_disarray(task_kids_loc,
                                           task_pars_loc,
                                           task_loc)
    return total_disarray

def find_least_disarrayed(mat, deps):
    '''Returns the rows of mat that have the
    lowest disarray score'''
    disarray_counts = np.zeros(mat.shape[0])
    for i, row in enumerate(mat):
        disarray_counts[i] = calc_disarray(row, deps)
    min_disarray = np.amin(disarray_counts)
    min_ind = np.where(disarray_counts==min_disarray)[0]
    return mat[min_ind,:]

def element_disarray(kid_loc, par_loc, loc):
    '''Returns a number according to how "out of place"
    an element is'''
    kids_to_left = np.extract(kid_loc<loc, kid_loc)
    pars_to_right = np.extract(par_loc>loc, par_loc)
    return (np.sum(loc - kids_to_left) + 
            np.sum(pars_to_right - loc))

def check_unique(mat, prev):
    '''Removes the rows in mat that
    are also in prev'''
    for i, row in enumerate(mat):
        if (prev==row).all(axis=1).any():
            mat[i,:] = np.zeros(
                mat.shape[1])
    mat = mat[~np.all(
        mat==0, axis=1)]
    return mat

def quick_best_order(arr, df, deps, count, prev):
    '''Returns an approximation for the optimal
    order.'''
    count += 1
    transposes = adj_transposes(arr)
    transposes = check_unique(transposes, prev)
    least_disarrayed = find_least_disarrayed(transposes, deps)
    expected_times = get_expected_times(least_disarrayed, df)
    best = least_disarrayed[np.argmin(expected_times)]
    prev = np.append(prev, np.expand_dims(best, 0), 0)
    if check_perm(best, deps)==True:
        return best
    elif count<50:
        return quick_best_order(best, df, deps, count, prev)
    else:
        return ("Error: Make sure dependencies don't produce"
                + " a contradiction.")

def present_list(arr, df):
    name_list = []
    index_to_name = dict(zip( df.index + 1, df['Task']))
    for index in arr:
        name_list.append(index_to_name[index])
    return pd.DataFrame(name_list, columns=["Task"])

    
def plot_fig(sorted_sums, hist_vals):
    fig, axs = plt.subplots(1, 2, figsize=(20, 8))
    axs[0].plot(np.arange(len(sorted_sums)), sorted_sums, color='black', ls='--')
    axs[0].set_xticks([])
    axs[0].set_ylabel('Time')
    axs[0].set_title('Expected time with respect to \n order of tasks')
    axs[1].hist(hist_vals, bins=30)
    axs[1].set_title('Approximate distribution of occurence of failure')
    axs[1].set_xlabel("Time")
    axs[1].set_yticks([])
    plt.show()

def plot_hist(trials):
    fig, axs = plt.subplots()

def simulate(df, index):
    totals = []
    prob = df['Probability'].values.astype(float)[index]
    time = df['Time'].values.astype(float)[index]
    lambdas = get_lambdas(time, prob)
    success_count = 10000
    prior_task_time = 0
    for j, rate in enumerate(lambdas):
        trials = RV(Exponential(rate=rate)).sim(success_count)
        failed = trials.filter_leq(time[j])
        totals.append(list(np.array(list(failed)) + prior_task_time))
        success = trials.filter_geq(time[j])
        success_count = len(success)
        prior_task_time += time[j]
    totals = flatten(totals)
    return totals

def get_lambdas(time, prob):
    return 1/time*np.log(1/(1 - prob))    

def flatten(int_list): 
    str_rep = str(list(int_list)).replace('[', '').replace(']', '')    
    return [float(x) for x in str_rep.split(',') if x.strip]

def main(df):
    df = switch_from_names(df)
    deps = df.filter(like='D').values
    times = df['Time'].values.astype(float)
    probs = df['Probability'].values.astype(float)
    srtd_rates = sort_rates(times, probs)
    prev = np.zeros([1,df.shape[0]])
    rows = df.shape[0]
    if rows < 10:
        perms = gen_perms(df)
        valid_perms = validate_perms(perms, deps)
        sums = get_expected_times(valid_perms, df)
        sorted_sums = np.sort(sums)
        best_order = valid_perms[np.argmin(sums)]
        best_df = present_list(best_order, df)
        best_index = best_df.replace(
            names_to_index_dict(df)).values.flatten() - 1
        hist_vals = simulate(df, list(best_index))
        printmd("## Report")
        printmd("##### The optimal order is ")
        display(best_df)
        printmd("##### the expected time you will work on this project is")
        printmd("##### " + str(sorted_sums[0]))
        printmd("##### and the probability you will succeed is")
        printmd("##### " + str(np.prod(1 - probs)) + ".")
        plot_fig(sorted_sums, hist_vals)

    else:
        best = quick_best_order(srtd_rates, df, deps, 0, prev)
        print('The optimal order is') 
        display(present_list(best, df))
        print("and the expected time is")
        display(get_expected_times(
                    np.expand_dims(best, 0), df)[0])

## Data Entry

How it works:
1. If you have previously used this app to build a sheet and you have the file saved, you can pick up where you left off by uploading it. Simply upload the file and click Build Sheet.
2. To start with a blank sheet, just click Build Sheet.
3. Fill the Tasks column with short names for each task, the Time column with your estimation for how long each task will take, and the Probability column with your estimation of the probability that each task will fail.
4. Let's say you put task A in row 1. You would then put the name of any tasks that you must complete before you can attempt task A in the D columns on row 1.
5. Click run to get a full report!
6. Click the link at the bottom of the report to download your sheet.

In [232]:
myupload = ipywidgets.FileUpload(accept='.csv', multiple=False)
display(myupload)

FileUpload(value={}, accept='.csv', description='Upload')

In [233]:
sheet_butt = ipywidgets.Button(description='Build Sheet')
out = ipywidgets.Output()
sheet = build_sheet()

def sheet_butt_func(_):
    global sheet
    with out:
        try:
            clear_output()
            uploaded_filename = next(iter(myupload.value))
            content = myupload.value[uploaded_filename]['content']
            with open('task_sheet.csv', 'wb') as f: f.write(content)
            sheet = build_sheet()
            display(sheet)
        except:
            sheet = build_sheet()
            display(sheet)
sheet_butt.on_click(sheet_butt_func)
display(ipywidgets.VBox([sheet_butt,out]))

In [236]:
run = ipywidgets.Button(description='Run')
out = ipywidgets.Output()
def run_program(_):
    with out:
        df = df_from_sheet(sheet)
        df.to_csv('task_sheet.csv', index=False)
        local_file = FileLink('./task_sheet.csv', result_html_prefix="Click here to download the sheet: ")
        return main(df), display(local_file)
        
run.on_click(run_program)
buttons = ipywidgets.HBox([run])
ipywidgets.VBox([buttons,out])

## Appendix A

We begin by letting $T$ be a random variable representing the time we spend on the project, from the begining to the point where we either succeed or fail. Our goal is to minimize $\text{E}[T]$.

Let $L$ be a random variable indicating the task we fail on. We define $L$ as

$$
L = \begin{cases} 
     i & \text{we fail on task }i \\
     0 & \text{we succeed}. \\ 
   \end{cases}
$$

We can now invoke the law of total expectation.

$$
\text{E}[T] = \text{E}[\text{E}[T \vert L]] = \text{P}(L = 0)\text{E}[T \vert L = 0] + \text{P}(L = 1)\text{E}[T \vert L = 1] + \dots + \text{P}(L = n)\text{E}[T \vert L = n].
$$

The user supplies us with their estimates of the probability of failure on each task, and their estimate for the time each task will take. We denote these as $p_1, \dots p_n$ and $t_1, \dots t_n$. For our purposes these are constant known values. It follows that

$$
\text{P}(L = 0) = (1 - p_1) \dots (1 - p_n) = \prod_{i = 1}^n (1 - p_i) \text{ and}
$$
$$
\text{E}[T \vert L = 0] = t_1 + \dots t_n = \sum_{i = 1}^n t_i.
$$
Take $ i \in [n]$. It's easy to see that
$$
\text{P}(L = i) = (1 - p_1) \dots (1 - p_{i - 1}) p_i = \prod_{k = 0}^ {i - 1} (1 - p_k)p_i.
$$

It is not obvious what $E[T \vert L_{i}]$ should be, and that is because we lack enough information about the distribution of $T$. We have to make an assumption, and the one we choose is that the probability of failing during the next minute of doing a task is independent of how long we have been doing the task (see <a href = https://cs.stanford.edu/~jsteinhardt/ResearchasaStochasticDecisionProcess.html>Research as a Stocastic Decision Process</a>). This implies that $f_T$ is given by

$$
f_T(t) = \begin{cases}
            \lambda_1 e^{-\lambda_1 t} & 0 \leq t < t_1 \\
            (1 - p_1) \lambda_2 e^{-\lambda_2 (t - t_1)} & t_1 \leq t < t_1 + t_2 \\
            \vdots & \vdots \\
           (1 - p_{i - 1}) \dots (1 - p_1) \lambda_i e^{-\lambda_i( t -  t_1 - \dots - t_{i - 1})} & t_1 + \dots + t_{i - 1} \leq t < t_1 + \dots + t_{i} \\
            \vdots & \vdots \\
            (1 - p_{n}) \dots (1 - p_{i - 1}) \dots (1 - p_1) \lambda_n e^{-\lambda_n (t - t_1 - \dots -t_{n - 1})} & t_1 + \dots t_{n - 1} \leq t < t_1  + \dots t_{n},
       \end{cases}
$$

where $\lambda_{i}$ is the failure rate of task $i$ given by
$$
\lambda_{i} = \frac{1}{t_i}\ln \Big{(}\frac{1}{1 - p_{i}}\Big{)}.
$$

From this we have
$$
f_{T \vert \{L = i\}}(t) = \frac{f_T(t)}{\text{P}(L = i)} = \frac{(1 - p_{i - 1}) \dots (1 - p_1) \lambda_i e^{-\lambda_i t -  t_1 - \dots - t_{i - 1}}}{(1 - p_{i - 1}) \dots (1 - p_1)p_i} = \frac{ \lambda_i e^{-\lambda_i t -  t_1 - \dots - t_{i - 1}}}{p_i} \text{ when } t_1 + \dots + t_{i - 1} \leq t < t_1 + \dots + t_{i}.
$$

Let $t_0 = t_1 + \dots + t_{i - 1}$. By performing a change of variables $ t \rightarrow t - t_0$ we have
$$
E[T \vert L = i] = \frac{1}{p_i} \int_0^{t_i} (t + t_0) \lambda_i e^{-\lambda_i t} dt,
$$

which after some simplification yields

$$
\text{E}[T \vert L = i] = \frac{1 + \lambda_i t_0 - e^{- \lambda_i t_i } - \lambda_i e^{ -\lambda_i t_i } (t_0 + t_i)}{ \lambda_i p_i}.
$$

Substituting in our value for $\lambda_i$ and simplifying we have

$$
\text{E}[T \vert L = i] = t_0 - \frac{t_i}{p_i} + \frac{t_i}{\ln \big{(} \frac{1}{1 - p_i} \big{)} } + t_i.
$$

Combining this with what we found for $\text{P}(L = i)$ and recalling that $t_0 = \sum_{j = 1}^{i - 1} t_j$ we have

$$
\text{P}(L = i)\text{E}[T \vert L = i] =  \prod_{k = 1}^ {i - 1} (1 - p_k)p_i \Bigg(\sum_{j = 0}^{i - 1} t_j - \frac{t_i}{p_i} + \frac{t_i}{\ln \big{(} \frac{1}{1 - p_i} \big{)} } + t_i \Bigg).
$$

Thus, our final expression for $ \text{E}[T]$ is

$$
\text{E}[T] = \prod_{i = 1}^n (1 - p_i)\sum_{i = 1}^n t_i + \sum_{i = 1}^n \Bigg( \prod_{k = 1}^ {i - 1} (1 - p_k)p_i \Bigg(\sum_{j = 0}^{i - 1} t_j - \frac{t_i}{p_i} + \frac{t_i}{\ln \big{(} \frac{1}{1 - p_i} \big{)} } + t_i \Bigg) \Bigg).
$$